In [ ]:
%load_ext autoreload
%autoreload 2
from awesome.run.awesome_config import AwesomeConfig
from awesome.run.awesome_runner import AwesomeRunner
from awesome.util.reflection import class_name
from awesome.analytics.result_model import ResultModel
from awesome.util.path_tools import get_project_root_path, get_package_root_path
import os
import torch
import re
from awesome.util.format import latex_postprocessor

os.chdir(get_project_root_path()) # Beeing in the root directory of the project is important for the relative paths to work consistently

In [ ]:
from awesome.analytics.result_comparison import ResultComparison

paths = ['./runs/unireps/joint','./runs/unireps/individual' ]
models = []

for path in paths:
    for folder in os.listdir(path):
        model = ResultModel.from_path(os.path.join(path, folder))
        models.append(model)

import re
p = r"#?(?P<cfg_num>\d+)?_?(?P<net>\w+)_benchmark(?P<feat>(\+\w+)*)\_(?P<date>\d{2}_\d{2}_\d{2})\_(?P<time>\d{2}_\d{2}_\d{2})"
pattern = re.compile(p)

for model in models:
    match = pattern.match(model.name)
    model_name = None
    feat = []
    if match:
        model_name = match.group('net')
        features = match.group('feat')
        if features is not None and features != "":
            feat = features.strip("+").split("+")
            if not any(["seed" in x for x in feat]):
                feat.append("seed42")
            feat = sorted(feat)
    else:
        print('No match for', model.name)
    model_name = model_name.replace("NET", "Net")
    model.display_name = model_name + " " + " ".join(feat)
    model.config.result_directory = "final_mask"
    model.save_config()


# Resort the models by name to get a meaningful table order

_order = []

models = sorted(models, key=lambda m: _order.index(m.name) if m.name in _order else 0)

comparison = ResultComparison(models)
comparison.assign_numbers(force=True)

os.environ['PLOT_OUTPUT_DIR'] = comparison.output_folder

save_args = dict(transparent=False, save=True, dpi=300, ext=["png", "pdf"])


In [ ]:
from typing import Any, Dict
import pandas as pd
model_names = set([m.config.name.split(" ")[0] for m in models])
features = set([y for m in models for y in m.config.name.split(" ")[1:]])
seeds = {x for x in features if "seed" in x}
features = features - seeds - {"convex", "joint"}
model_names, features, seeds

columns = list(features) + ['joint ' + x for x in features]
columns = columns + [x + " seeds" for x in columns]

rows = list(model_names)

df = pd.DataFrame(index=rows, columns=columns)
df[pd.isna(df)] = 0

for c in [x for x in df.columns if "seeds" in x]:
    df.loc[:, c] = ""

def extract_features(model: ResultModel) -> Dict[str, Any]:
    res = dict()
    
    res['joint'] = "joint" in model.config.name
    
    res['model_name'] = model.config.name.split(" ")[0]
    model_features = model.config.name.split(" ")[1:]
    
    if res['joint']:
        model_features.remove("joint")
    
    seed = [x for x in model_features if "seed" in x][0]
    model_features.remove(seed)

    res['seed'] = int(seed.replace("seed", ""))

    if "convex" in model_features:
        model_features.remove("convex")
        res['convex'] = True
    else:
        res['convex'] = False

    assert len(model_features) == 1, f"Multiple features {model_features} in model {model.output_folder}"
    res['feature_type'] = model_features[0]

    return res


for model in models:
    res = extract_features(model)

    is_joint = res['joint']
    model_name =  res['model_name']
    model_features = model.config.name.split(" ")[1:]
    
    feat_type = "joint " + res['feature_type'] if is_joint else res['feature_type']   
    df.loc[model_name, feat_type] += 1
    seed_col = feat_type + " seeds"
    df.loc[model_name, seed_col] = str(df.loc[model_name, seed_col]) + " " + str(res['seed'])
    

df

In [ ]:
from typing import Tuple
import pandas as pd

metrics = [
    "eval/epoch/MeanForegroundBinaryMIOU" ,
    "eval/epoch/MeanPixelAccuracy",
    "eval/epoch/MeanPriorPixelAccuracy",
    "eval/epoch/MeanPriorForegroundBinaryMIOU"
]

col_mapping = {
    "eval/epoch/MeanForegroundBinaryMIOU": "mIoU",
    "eval/epoch/MeanPixelAccuracy": "Acc.",
    "eval/epoch/MeanPriorPixelAccuracy" : "Convex Acc.",
    "eval/epoch/MeanPriorForegroundBinaryMIOU": "Convex mIoU"
}

df = comparison.metric_table(metrics, 
                             ref="last", 
                             mode="max",
                        formatting=False)

df = df.reset_index()

def extract_ft_row(row: pd.Series) -> Tuple[str, bool, str, int]:
    name = row['index']
    model = [m for m in models if m.name == name][0]
    res = extract_features(model)
    return (res['model_name'], res['joint'], res['feature_type'], res['seed'])


df[['model_name', 'joint', 'feature_type', 'seed']] = df.apply(extract_ft_row, axis=1, result_type="expand")

grouped = df.groupby(['model_name', 'joint', 'feature_type'])
display(df)

In [ ]:
sub_select_df = df[(df['seed'] == 42) & (df['feature_type'] == "feat") & (df['model_name'] == 'FCNet')]

model_names = sub_select_df['index'].tolist()
model_names

In [ ]:
# TODO Recompute the mIoU for the final masks
import numpy as np
from awesome.agent.torch_agent_checkpoint import TorchAgentCheckpoint
from awesome.agent.util.tracker import Tracker
from awesome.measures.miou import MIOU
from awesome.measures.pixel_accuracy import PixelAccuracy
import awesome.run.functions as f
from awesome.util.temporary_property import TemporaryProperty
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex, to_rgb

model = models[0]
plot_models = [m for m in models if m.name in model_names] #[model] # models

miou = MIOU(invert=True)
acc = PixelAccuracy()

tab10_blue = to_hex(plt.get_cmap("tab10")(0))
tab10_orange = to_hex(plt.get_cmap("tab10")(1))
tab10_red = to_hex(plt.get_cmap("tab10")(2))
tab10_green = to_hex(plt.get_cmap("tab10")(3))

@f.saveable()
def plot_scribbles(image, fg, bg, tight: bool =False, size: float=5):
    fig = plt.figure(figsize=(size, size))
    fig = None
    ax = None
    if tight:
        sizes = np.shape(image)[1:]  
        fig = plt.figure(figsize=(size, size))
        dpi = 300
        fig.set_size_inches(size * (sizes[1] / dpi), size * (sizes[0] / dpi), forward = False)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
    else:
        fig, ax = plt.subplots(1, 1, figsize=(size, size))

    ax.imshow(image.permute(1, 2, 0))

    #ax.pcolormesh(subsampled_image, edgecolors="#a2a6ab", linewidth=.5, facecolors='none',rasterized=True, zorder=0)

    fg_pred_color = np.zeros(fg.shape + (4,), dtype=np.float32)

    fg_pred_color[..., 0:3] = to_rgb(tab10_green)
    alpha_fg_scribble = 1
    fg_pred_color[..., 3] = fg * alpha_fg_scribble

    ax.imshow(fg_pred_color)

    bg_pred_color = np.zeros(bg.shape + (4,), dtype=np.float32)

    bg_pred_color[..., 0:3] = to_rgb(tab10_red)
    alpha_bg_scribble = 1
    bg_pred_color[..., 3] = bg * alpha_bg_scribble

    ax.imshow(bg_pred_color)
    return fig

text = []

def to_text(*txt: str):
    if isinstance(txt, (list, tuple)):
        txt = "".join(txt)
    print(txt)
    text.append(txt)

for model in plot_models:
    model: ResultModel
    with TemporaryProperty(model, getitem_mask_mode="both"):
        runner = model.get_runner(-1)
        tracker: Tracker = model.get_tracker(-1)
        dataloader = runner.dataloader
        with TemporaryProperty(dataloader, dimension="3d"), plt.ioff():
            indices = [25]
            
            for image_idx in indices:
                image, ground_truth, _input, target, fg, bg, prior_state = f.prepare_input_eval(dataloader, None, image_idx)
                fg = fg.reshape(image.shape[1:])
                bg = bg.reshape(image.shape[1:])
                ground_truth = 1. - ground_truth
                predicted_mask, prior_mask = model[image_idx]

                model_name = model.name.replace(" ", "_")
                to_text("Model: " + model_name)
                to_text("Image: " + str(image_idx))

                fig = plot_scribbles(image, bg, fg, path=f"notebooks/temp/scrib_{model_name}_{image_idx}.png", save=True, size=5, tight=True, override=True)
                display(fig)
                
                fig = f.plot_mask(image, predicted_mask, path=f"notebooks/temp/seg_{model_name}_{image_idx}.png", save=True, size=5, tight=True, color=tab10_blue, override=True)
                #to_text(f"SEG MIOU: ", "{:.3f}".format(miou(predicted_mask, ground_truth)), " Acc.: ", "{:.3f}".format(acc(predicted_mask, ground_truth)))
                display(fig)
                
                fig = f.plot_mask(image, prior_mask, path=f"notebooks/temp/prior_{model_name}_{image_idx}.png", save=True, size=5, tight=True, color=tab10_orange, override=True)
                #to_text(f"Prior MIOU: ", "{:.3f}".format(miou(prior_mask, ground_truth)), " Acc.: ", "{:.3f}".format(acc(prior_mask, ground_truth)))
                display(fig)

with open("./notebooks/temp/qualitative_measures.txt", "w") as f:
    f.write("\n".join(text))

print("\n".join(text))

In [ ]:
comparison.open_folder()

In [ ]:
model.config.name